# SpaceMissionDES Demonstration
#### CSE 6730, Spring 2022, Project Group #25
Authors:
- Benjamin Merrel
- David Gomez
- Edmund Chen

#### Load Dependencies

In [6]:
# Standard Library
import logging
import asyncio
from pathlib import Path
# Dependencies
from scipy.stats import *
import pandas as pd
import altair as alt
alt.data_transformers.enable('data_server')
# SpaceMissionDES
from objects.events import *
from objects.activities import *
from objects.vehicles import Vehicle
from objects.predicates import Predicate, vehicle_in_activity
from drivers.simulator import Simulator
from utilities.logging import set_logging_level

## Motivation

Space missions are often expressed as a complicated sequence of events.  We call this a *concept of operations* or a **ConOps**.


<img src="./media/apollo_flightdiagram.jpg">

Stuff about how space missions are getting more complicated and now involed far more interacting entities.

Something about how discrete event simulation is a natural fit -- maybe tie back to Grant Cate's early justifications?

## Problem Definition

- Choosing between a large number of alternative ConOps
- Figuring out which decisions or parts of a mission matter the most
- Evaluating a mission in terms of probability of success, but also caring about schedule
- Complicated range of potential states, and its just natural to use a DES

Introduce the case of a Mars mission with on-orbit aggregation
- Note that the number of tanker flights is a deign parameter. 
- Trading the number of flights and the capability of each tanker is tied to cost, schedule, and reliability.
- We can address two of those with Discrete event simulation

## Approach

### Discrete Event Simulation
We decided to build a process-oriented discrete event simulation.
- The mental model of a string of interdependent processes and events that connect them maps well to space mission.
- Many space missions effectively assume discrete time steps by treating the dynamic events (i.e. big burns) as instantaneous and book-ended by quiescent transfers.
    - Discrete Time and Discrete state
    - Limited number of types of processes and events lends itself to DES building blocks

### General DES Overview
Reminder of how a discrete event simulation works

### Tailoring DES for Space Missions
Events, Activities, Vehicles, and Predicates

### Case Study: Mars Transfer Vehicle with On-Orbit Aggregation

Why

<img src="./media/Tanker-Flow-Diagram.png">

### DES Implementation

Top Level Parameters

In [2]:
# Probabilistic Inputs
pra = {
    "scrub": 1/4,
    "ascent": 1/100,
    "RPO": 1/500,
    "mps_burn": 1/200,
    "dock": 1/200,
    "checkout": 1/1000
}

# Fleet Management and Requirements
N_tankers_available =  1
N_transfers_required = 6

initial_vehicles = []

#### MTV ConOps Definition

In [3]:
# MTV - Mars Transfer Vehicle

# Events
INIT = Event("INIT")
launch = Event("launch")
burnout = Event("burnout")
capture = Event("capture")
filled = Event("filled")
tmi_burn = Event("tmi_burn")
moi_burn = Event("moi_burn")
ARRIVE = Completor("ARRIVE")
DONE = Completor("DON")

scrub = Event("scrub")

# Predicates
def check_transers(p, sim):
    # Check on the tanker
    tanker = sim.entities["Tanker"]
    if tanker.state["transfers"] < N_transfers_required:
        return False
    else:
        logging.info(f"Predictate <{p.predicate.name}> Satisfied")
        return True

until_N_transfers = Predicate(f"Wait until {N_transfers_required} propellant transfers are completed", check_transers)

# ConOps
conops_MTV = ConOps({

    # Nominal
    INIT.name:     Activity("Countdown", INIT, launch, duration=3, p_fail=pra["scrub"], failure=scrub),
    launch.name:   Activity("Ascent", launch, burnout, duration=1, p_fail=pra["ascent"]),
    burnout.name:  Activity("Orbit Insertion", burnout, capture, duration=2, p_fail=pra["mps_burn"]),
    # capture.name:  Activity("Propellant Aggregation", capture, filled, duration = 100),
    capture.name:  PredicatedActivity("Propellant Aggregation", capture, filled, predicate=until_N_transfers),
    filled.name:   Activity("Final Checkout", filled, tmi_burn, duration=2, delay=weibull_min(c=0.5, loc=0, scale=0.1), p_fail=pra["checkout"]),  #TODO: add check to see if clock is < critical value
    tmi_burn.name: Activity("Begin Mars Transit", tmi_burn, DONE, duration=0, p_fail=pra["mps_burn"]),
    # Contigencies
    scrub.name: Activity("Recycle", scrub, INIT, duration=0, delay=weibull_min(c=1, loc=0, scale=4)),
})

initial_vehicles += [
    (0.0, Vehicle("MTV", conops_MTV))
]

#### Tanker ConOps Definition

In [4]:
# -----------------------------------------------------------------------------------------------------------------
# Tanker

# Events
begin = Event("begin")
approach = Event("approach")
dock = Event("dock")
undock = Event("undock")
land = Event("land")

spare = Event("spare")
no_more_spares = Event("no_more_spares")

# Predicates
after_MTV_deploy = Predicate("Wait for MTV", vehicle_in_activity(vehicle="MTV", activity="Propellant Aggregation"))

# Branching Events
def limit_spares(sim, vehicle):          # branching logic functions always take (sim, vehicle)
    if vehicle.state["fleet"] > 0:
        return begin
    else:
        return no_more_spares

limit_tanker_spares = Branch(f"Limit to {N_tankers_available-1} Spares", logic=limit_spares)


def count_transfers(sim, vehicle):          # branching logic functions always take (sim, vehicle)
    if vehicle.state["transfers"] < N_transfers_required:
        return begin
    else:
        return land

until_n_tankers = Branch(f"Until {N_transfers_required} Transfers", logic=count_transfers)

# ConOps
conops_Tanker = ConOps({

    # Nominal
    INIT.name:     PredicatedActivity("Wait for MTV Deploy", INIT, limit_tanker_spares, predicate=after_MTV_deploy),

    begin.name:    Activity("Countdown", begin, launch, duration=0, p_fail=pra["scrub"], failure=scrub),
    launch.name:   Activity("Ascent", launch, burnout, duration=1, p_fail=pra["ascent"], failure=spare, update={'flights': 1}),
    burnout.name:  Activity("Orbit Insertion", burnout, capture, duration=2, p_fail=pra["mps_burn"], failure=spare),
    capture.name:  Activity("Redezvous", capture, approach, duration = 1, p_fail=pra["RPO"]),
    approach.name: Activity("RPOD", approach, dock, duration = 1, p_fail=pra["dock"]),
    dock.name:     Activity("Prop Transfer", dock, undock, duration=0.5, update={'transfers': 1}),  # TODO: add delay
    undock.name:   Activity("Return to Base", undock, until_n_tankers, duration = 1, p_fail=pra["dock"]), # TODO: add restart conditions
    land.name:     Activity("End Tanker Mission", land, DONE, duration=0),

    # Contigencies
    scrub.name:    Activity("Recycle", scrub, INIT, duration=0, delay=weibull_min(c=1, loc=0, scale=4)),
    spare.name:    Activity("Prepare Spare", spare, INIT, duration=10, delay=weibull_min(c=0.8, loc=0, scale=5), update={'fleet': -1}),
    no_more_spares.name: Activity("Out of Spare Tankers", no_more_spares, Failure(), duration=0)
})

initial_vehicles += [
    (0.0, Vehicle("Tanker", conops_Tanker, state={'flights':0, 'transfers':0, 'failures':0, 'fleet': N_tankers_available}))
]

#### Mission Demonstration

In [5]:
set_logging_level(logging.INFO)

sim = Simulator()

try:
    await sim.run(initial_vehicles)
except asyncio.CancelledError:
    logging.warning("CONOPS FAILED")


	EVENT:  INIT  @ time 0.00
	  VEHICLE MTV > Begin ACTIVITY:  Countdown

	EVENT:  INIT  @ time 0.00
	  VEHICLE Tanker > Begin ACTIVITY:  Wait for MTV Deploy

	EVENT:  launch  @ time 3.00
	  VEHICLE MTV > Begin ACTIVITY:  Ascent

	EVENT:  burnout  @ time 4.00
	  VEHICLE MTV > Begin ACTIVITY:  Orbit Insertion

	EVENT:  capture  @ time 6.00
	  VEHICLE MTV > Begin ACTIVITY:  Propellant Aggregation
Predictate <Wait for MTV> Satisfied

	EVENT:  begin  @ time 6.00
	  VEHICLE Tanker > Begin ACTIVITY:  Countdown
WARNING>	  FAIL -- VEHICLE Tanker failed ACTIVITY:  Countdown

	EVENT:  scrub  @ time 6.00
	  VEHICLE Tanker > Begin ACTIVITY:  Recycle

	EVENT:  INIT  @ time 7.38
	  VEHICLE Tanker > Begin ACTIVITY:  Wait for MTV Deploy
Predictate <Wait for MTV> Satisfied

	EVENT:  begin  @ time 7.38
	  VEHICLE Tanker > Begin ACTIVITY:  Countdown

	EVENT:  launch  @ time 7.38
	  VEHICLE Tanker > Begin ACTIVITY:  Ascent

	EVENT:  burnout  @ time 8.38
	  VEHICLE Tanker > Begin ACTIVITY:  Orbit Insertion


### Gaining Insight via Monte Carlo Studies

Something about how the real value comes from running a monte carlo

Mention that in our case, we found that running muliple replicationsin parallel was sufficient -- no need for additional sophistication.

Make a little graphic to show the monte carlo flow

#### Convergence Study

*How many replicants do we need?*

<img src="./media/Convergence_Case04-r05__Tx02-Tk10.svg">

#### Processing Utilities

In [74]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

#### Compare Cases

##### Load & Format Data

In [52]:
# Load the data
data_dir = Path("/Users/ben/Developer/SpaceMissionDES/results")

case_dirs = [
    data_dir / "Case04-r05__Tx15-Tk01",
    data_dir / "Case04-r05__Tx15-Tk02",
    data_dir / "Case04-r05__Tx15-Tk04",
]

raw_cases = []
for path in case_dirs:
    # Load the raw data to dataframe
    case_results = pd.read_csv(path / "mc.csv")
    # Parse the case inputs
    case_details = path.name.split("__")[1]
    tx_required  = int(case_details.split('-')[0].strip('Tx'))
    tk_available = int(case_details.split('-')[1].strip('Tk'))
    # Assign new flags
    case_results["Case"] = case_details
    case_results["Tx_Required"] = tx_required
    case_results["Fleet_Size"] = tk_available
    
    raw_cases.append(case_results)

results = pd.concat(raw_cases).reset_index(drop=True)
results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity,Case,Tx_Required,Fleet_Size
0,1,False,32.541477,2,19.000000,Tanker,Ascent,Tx15-Tk01,15,1
1,4,False,31.827839,2,12.500000,Tanker,Countdown,Tx15-Tk01,15,1
2,4,False,31.827839,3,31.827839,Tanker,RPOD,Tx15-Tk01,15,1
3,7,False,62.338018,2,6.000000,Tanker,Countdown,Tx15-Tk01,15,1
4,7,False,62.338018,3,34.327839,Tanker,Ascent,Tx15-Tk01,15,1


Address Case Summary Data

In [57]:
case_summaries = []

for case in results.Case.unique():

    df = (
        results[results.Case==case]
        .groupby("replicant", as_index=False)
        .first()
        .loc[:, ["Case", "replicant", "outcome", "duration"]]
    )
    case_summaries.append(df)

summaries = pd.concat(case_summaries).reset_index(drop=True)
summaries.head()

,Case,replicant,outcome,duration
0,Tx15-Tk01,0,True,135.989214
1,Tx15-Tk01,1,False,32.541477
2,Tx15-Tk01,2,False,117.734047
3,Tx15-Tk01,3,True,151.230467
4,Tx15-Tk01,4,False,31.827839


Analyze Case CDFs

In [75]:
data = []

for case in df.Case.unique():

    sample = df[df.Case==case].duration.values

    qe, pe = ecdf(sample)
    case_data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    case_data["Case"] = case

    data.append(case_data)

cdf_data = pd.concat(data).reset_index(drop=False)
cdf_data.Case.unique()

array(['Tx15-Tk01', 'Tx15-Tk02', 'Tx15-Tk04'], dtype=object)

##### Single Case

##### Multiple Cases

In [ ]:
df = summaries
df.loc[df.outcome == False, "duration"] = 1e6

alt.Chart(df).mark_bar().encode(
    # alt.X("duration:Q", bin=True),
    alt.X("duration:Q", bin=alt.Bin(extent=[80, 220], step=5)),
    y='count()',
    row = "Case",
    color="Case"
).properties(
    width = 600,
    height = 100
)

alt.Chart(...)

%d %s %.2fms


INFO:tornado.access:304 GET /87c6cb76ff6c2d5d32e5b4c297e71780.json (::1) 1.38ms


In [81]:
fig = alt.Chart(cdf_data).mark_line().encode(
    x = alt.X("duration", scale=alt.Scale(domain=(80,220))),
    y = "prob",
    color = "Case"
).properties(width=500, height=250)

fig.interactive()

alt.Chart(...)

%d %s %.2fms


INFO:tornado.access:304 GET /ecdca40ff5f81ef134e33b276b8cbf06.json (::1) 0.31ms


### Parameters Sweep

How many tankers for alternate designs?

In [86]:
# Load the data
data_dir = Path("/Users/ben/Developer/SpaceMissionDES/results/Parameter_Sweep")

# case_dirs = [
#     data_dir / "Case04-r05__Tx15-Tk01",
#     data_dir / "Case04-r05__Tx15-Tk02",
#     data_dir / "Case04-r05__Tx15-Tk04",
# ]

raw_cases = []
for path in data_dir.iterdir():
    try:
        # Load the raw data to dataframe
        case_results = pd.read_csv(path / "mc.csv")
        # Parse the case inputs
        case_details = path.name.split("__")[1]
        tx_required  = int(case_details.split('-')[0].strip('Tx'))
        tk_available = int(case_details.split('-')[1].strip('Tk'))
        # Assign new flags
        case_results["Case"] = case_details
        case_results["Tx_Required"] = tx_required
        case_results["Fleet_Size"] = tk_available
        
        raw_cases.append(case_results)
    except NotADirectoryError:
        continue

results = pd.concat(raw_cases).reset_index(drop=True)
results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity,Case,Tx_Required,Fleet_Size
0,4,False,9.000000,2,9.000000,Tanker,Redezvous,Tx05-Tk08,5,8
1,0,True,39.505085,0,NaN,NaN,NaN,Tx05-Tk08,5,8
2,2,True,57.014809,2,6.000000,Tanker,Countdown,Tx05-Tk08,5,8
3,2,True,57.014809,3,19.901978,Tanker,Countdown,Tx05-Tk08,5,8
4,6,True,60.014809,2,0.000000,MTV,Countdown,Tx05-Tk08,5,8


In [87]:
results.Case.unique()

array(['Tx05-Tk08', 'Tx05-Tk06', 'Tx05-Tk01', 'Tx10-Tk10', 'Tx10-Tk04',
       'Tx10-Tk02', 'Tx15-Tk01', 'Tx15-Tk06', 'Tx15-Tk08', 'Tx15-Tk10',
       'Tx05-Tk02', 'Tx05-Tk04', 'Tx05-Tk10', 'Tx10-Tk01', 'Tx10-Tk06',
       'Tx10-Tk08', 'Tx15-Tk04', 'Tx15-Tk02'], dtype=object)